# Marisa Trie Implementation

In [6]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

Matplotlib is building the font cache; this may take a moment.


In [ ]:
class PatriciaTrieNode():
    def __init__(self, key=""):
        self.key = key
        self.children = {}
        self.is_end_of_word = False

class PatriciaTrie():
  def  __init__(self):
    self.root = PatriciaTrieNode()
  def insert(self, word):
    current = self.root
    while word:
      for key in list(current.children.keys()):
        common_prefix_length = self._common_prefix_length(word, key)
        if common_prefix_length > 0:
          if common_prefix_length < len(key):
            existing_node = current.children.pop(key)
            new_node = PatriciaTrieNode(key[:common_prefix_length])
            current.children[new_node.key] = new_node
            new_node.children[key[common_prefix_length:]] = existing_node
            existing_node.key = key[common_prefix_length:]

            if common_prefix_length == len(word):
              new_node.is_end_of_word = True
            else:
              new_node.children[word[common_prefix_length:]] = PatriciaTrieNode(word[common_prefix_length:])
              new_node.children[word[common_prefix_length:]].is_end_of_word = True
            return
            word = word[common_prefix_length:]
            current = current.children[key]
            break
        else:
          current.children[word] = PatriciaTrieNode(word)
          current.children[word].is_end_of_word = True
          return

  def search(self, word):
    current = self.root
    while word:
      found = False
      for key, node in current.children.items():
        if word.startswith(key):
          if len(key) == len(word):
            return node.is_end_of_word
          word = word[len(key):]
          current = node
          found = True
          break
      if not found:
        return False
    return current.is_end_of_word

  def delete(self, word):
    def _delete(node, word):
      if not word:
        if not node.is_end_of_word:
          return False
        node.is_end_of_word = False
        return len(node.children) == 0

      for key, child in list(node.children.items()):
        if word.startswith(key):
          if _delete(child, word[len(key):]):
            del node.children[key]
            return not node.is_end_of_word and len(node.children) == 0
          return False
      return False

    _delete(self.root, word)

  def _common_prefix_length(self, str1, str2):
    i = 0
    while i < len(str1) and i < len(str2) and str1[i] == str2[i]:
      i += 1
    return i

## Patricia Tree Attempt 2

In [8]:
class PatriciaTrieNode():
    def __init__(self, key=""):
        self.key = key
        self.children = {}
        self.is_end_of_word = False

class PatriciaTrie():
    def __init__(self):
        self.root = PatriciaTrieNode()

    def insert(self, word):
        current = self.root
        while word:
            found = False
            for key in list(current.children.keys()):
                common_prefix_length = self._common_prefix_length(word, key)
                
                if common_prefix_length > 0:
                    # Case 1: If the common prefix is shorter than the existing key, split the node
                    if common_prefix_length < len(key):
                        existing_node = current.children.pop(key)
                        new_node = PatriciaTrieNode(key[:common_prefix_length])
                        current.children[new_node.key] = new_node
                        new_node.children[key[common_prefix_length:]] = existing_node
                        existing_node.key = key[common_prefix_length:]

                    # Case 2: Update current node to handle the new word
                    current = current.children[key[:common_prefix_length]]
                    word = word[common_prefix_length:]

                    # Case 3: If the word length matches, mark it as an end of word
                    if not word:
                        current.is_end_of_word = True
                    else:
                        # Create a new node for the remaining part of the word
                        if word not in current.children:
                            current.children[word] = PatriciaTrieNode(word)
                            current.children[word].is_end_of_word = True
                    return
                
            # Case 4: If no common prefix, add the word as a new child
            current.children[word] = PatriciaTrieNode(word)
            current.children[word].is_end_of_word = True
            return

    def search(self, word):
        current = self.root
        while word:
            found = False
            for key, node in current.children.items():
                if word.startswith(key):
                    if len(key) == len(word):
                        return node.is_end_of_word
                    word = word[len(key):]
                    current = node
                    found = True
                    break
            if not found:
                return False
        return current.is_end_of_word

    def delete(self, word):
        def _delete(node, word):
            if not word:
                if not node.is_end_of_word:
                    return False
                node.is_end_of_word = False
                return len(node.children) == 0

            for key, child in list(node.children.items()):
                if word.startswith(key):
                    if _delete(child, word[len(key):]):
                        del node.children[key]
                        return not node.is_end_of_word and len(node.children) == 0
                    return False
            return False

        _delete(self.root, word)

    def _common_prefix_length(self, str1, str2):
        i = 0
        while i < len(str1) and i < len(str2) and str1[i] == str2[i]:
            i += 1
        return i

Alan

# Plan
1. Implement the Patricia Trie
2. Compare how it stores a single genome (GRCh38) in terms of size
    - potentially explore size compression of input strings here?
3. Compare the time tradeoff to construct the sketch
4. Compute phylogenetic distance between two sketches



## Tools to Compare against
- Python dictionary -> naive
- Python implementation of trie
- Python implementation of Marisa trie
- Python implementation of radix trie



In [ ]:
# prompt: download grch38 genome from ftp into google drive

!pip install wget

import os
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Create a directory in Google Drive to store the genome
drive_dir = "/content/drive/MyDrive/GRCh38"  # Change this to your desired directory
if not os.path.exists(drive_dir):
    os.makedirs(drive_dir)

# Download the GRCh38 genome (replace with the actual FTP link)
# Example using wget:
# You'll need to find the correct FTP link to the GRCh38 genome files you need.
# This example assumes you want to download the chromosome 1 fasta file.
# Replace with the appropriate URL
!wget ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/001/405/GCF_000001405.39_GRCh38.p13/GCF_000001405.39_GRCh38.p13_genomic.fna.gz -O "/content/drive/MyDrive/GRCh38/GRCh38_chr1.fna.gz"

# You'll likely want to download multiple files. Adapt this to loop through the needed files and save them properly
# Example:
# for i in range(1, 23):
#   !wget ftp://<correct ftp address for chromosome {i}> -O "/content/drive/MyDrive/GRCh38/GRCh38_chr{i}.fna.gz"
# # Download other genome files as needed
# ...

print(f"Genome files downloaded to: {drive_dir}")

# Unzip (if needed):
!gunzip "/content/drive/MyDrive/GRCh38/GRCh38_chr1.fna.gz"

  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9656 sha256=97b1602c32e28c5ed968d731022f8c1c03b62e4c11b23983d03a6530ab00176f
  Stored in directory: /root/.cache/pip/wheels/8b/f1/7f/5c94f0a7a505ca1c81cd1d9208ae2064675d97582078e6c769
Successfully built wget
Mounted at /content/drive
--2024-11-08 21:49:15--  ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/001/405/GCF_000001405.39_GRCh38.p13/GCF_000001405.39_GRCh38.p13_genomic.fna.gz
           => ‘/content/drive/MyDrive/GRCh38/GRCh38_chr1.fna.gz’
Resolving ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)... 130.14.250.13, 130.14.250.31, 130.14.250.7, ...
Connecting to ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)|130.14.250.13|:21... connected.
Logging in as anonymous ... Logged in!
==> SYST ... done.    ==> PWD ... done.
==> TYPE I ... done.  ==> CWD (1) /genomes/all/GCF/000/001/405/GCF_000001405.39_GRCh38.p13 ... done.
==> SIZE GCF_000001405.39_GRCh38.p13_genomic.fna.gz ... 964894122
==

In [ ]:
!pip install pyfaidx
from pyfaidx import Fasta
!pip install tqdm
from tqdm import tqdm

In [ ]:
# loading k-mers from index into the sketch
def build_kmer_index(genome_file, obj, k):
    """
    Reads a genome file, extracts all k-mers of length k, and adds them to the given object.

    Parameters:
    genome_file (str): Path to the genome file in FASTA format.
    obj (object): An object with `insert`, `search`, and `delete` methods.
    k (int): Length of k-mer to extract and add to the object.
    """
    # Open the genome file with pyfaidx
    genome = Fasta(genome_file)

    # Loop over each chromosome or sequence in the genome file
    for record in genome:
        sequence = str(record)  # Get the sequence as a string
        seq_len = len(sequence) - k + 1  # Number of k-mers in this sequence

        # Use tqdm to show progress for the k-mer extraction
        for i in tqdm(range(seq_len), desc=f"Processing {record.name}"):
            kmer = sequence[i:i + k]
            obj.insert(kmer)

    print(f"All {k}-mers have been added to the object.")

In [ ]:
## Naive Approach
class NaiveSketch():
    def __init__(self):
        self.dict = {}

    def insert(self, word):
        self.dict[word] = self.dict.get(word, 0) + 1

    def search(self, word):
        return self.dict[word]

    def delete(self, word):
        self.dict[word] = self.dict.get(word, 1) - 1

In [ ]:
genome_file = "/content/drive/MyDrive/GRCh38/GRCh38_chr1.fna"  # Path to your GRCh38 genome file
k = 21  # Set your desired k-mer length

# Create an instance of the Naive dict
naive = NaiveSketch()

# Build the k-mer index using the naive
build_kmer_index(genome_file, naive, k)

NameError: name 'NaiveSketch' is not defined

Amy

Emma

Renee